In [1]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

tf.random.set_seed(777)

2.3.1


In [2]:
# 8개의 데이터, 4개의 feature
x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
# 8개의 feature
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]
print(x_data)
# convert int numpy and float format
x_data = np.asarray(x_data, dtype = np.float32)
y_data = np.asarray(y_data, dtype = np.float32)

[[1, 2, 1, 1], [2, 1, 3, 2], [3, 1, 3, 4], [4, 1, 5, 5], [1, 7, 5, 5], [1, 2, 5, 6], [1, 6, 6, 6], [1, 7, 7, 7]]


In [3]:
#@@@@@@@@@화긴~~~~~@@@@@@@@@@@@
x_data
# 결과를 보면 1. 해서 float형으로 바뀐 것을볼 수 있다.

array([[1., 2., 1., 1.],
       [2., 1., 3., 2.],
       [3., 1., 3., 4.],
       [4., 1., 5., 5.],
       [1., 7., 5., 5.],
       [1., 2., 5., 6.],
       [1., 6., 6., 6.],
       [1., 7., 7., 7.]], dtype=float32)

In [4]:
# dataset을 선언합니다.
nb_classes = 3 #class의 개수
# 

print(x_data.shape)
print(y_data.shape)

(8, 4)
(8, 3)


In [5]:
# Weight, bias를 setting하는 것
W = tf.Variable(tf.random.normal((4, nb_classes)), name = 'weight')
    # 4개의 특징과 3개의 클래스를 정의
    # random.normal 값으로 랜덤값을 정해줌
b = tf.Variable(tf.random.normal((nb_classes,)), name = 'bias')
variables =[W,b]
    # variable으로 저장함
    
# Weight, bias값을 출력해본다.
print(W)
print(b)

<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[ 0.7706481 ,  0.37335402, -0.05576323],
       [ 0.00358377, -0.5898363 ,  1.5702795 ],
       [ 0.2460895 , -0.09918973,  1.4418385 ],
       [ 0.3200988 ,  0.526784  , -0.7703731 ]], dtype=float32)>
<tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([-1.3080608 , -0.13253094,  0.5513761 ], dtype=float32)>


In [6]:
# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def hypothesis(X):
    return tf.nn.softmax(tf.matmul(X, W) + b)

print(hypothesis(x_data))

tf.Tensor(
[[1.36571955e-02 7.90162385e-03 9.78441238e-01]
 [3.92597765e-02 1.70347411e-02 9.43705440e-01]
 [3.80385250e-01 1.67723164e-01 4.51891571e-01]
 [3.23390484e-01 5.90759404e-02 6.17533624e-01]
 [3.62997412e-06 6.20727292e-08 9.99996305e-01]
 [2.62520202e-02 1.07279615e-02 9.63019967e-01]
 [1.56525111e-05 4.21802781e-07 9.99983907e-01]
 [2.94076904e-06 3.81133276e-08 9.99997020e-01]], shape=(8, 3), dtype=float32)


In [7]:
# softmax값 확인해보기
sample_db = [[8,2,1,4]]
sample_db = np.asarray(sample_db, dtype = np.float32)

print(hypothesis(sample_db))
# 결과를 보면 첫번째 값이 가장 큰 것을 확인할 수 있음.
# 결과값 세개 모두 더하면 1이 나옴. [4개]가 들어가면 a,b,c중 하나 고르는거 softmax 함수로

tf.Tensor([[0.9302204  0.06200533 0.00777428]], shape=(1, 3), dtype=float32)


In [8]:
#cost 구하는 함수
def cost_fn(X, Y):
    logits = hypothesis(X)
        # softmax 함수를 적용한 다음
    cost = -tf.reduce_sum(Y *tf.math.log(logits), axis = -1)
        # cross entropy값을 이용해 구함 (Y*log(y_hat))
    cost_mean = tf.reduce_mean(cost)
        # 그 다음 cost의 평균을 구함
    return cost_mean

print(cost_fn(x_data, y_data))
# cost의 최종 평균이 출력됨

tf.Tensor(6.07932, shape=(), dtype=float32)


In [9]:
# 이건 y=x^2 가 있을 때 y에 대해서 x를 미분했을때, 3일때 y값을 구하는거
x =  tf.constant(3.0)
with tf.GradientTape() as g:
    g.watch(x)
    y = x*x  # x^2
dy_dx = g.gradient(y,x)
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)


In [33]:
# cost함수를 최적화 하기 위해 경사하강법을 사용함
# 경사하강법 (Gradient Function)을 구현해보자.

def counter():
    i = 0
    def count():
        nonlocal i
        i += 1
        return i
    return count

c = counter()
# 참고로 with - as 함수는 문해프 마지막단원 파일처리할 때 많이 배웠었음
# GradientTape()는 context안에서 실행되는 모든 연산을 테이프에 '기록'함
# 그 다음 텐서플러오눈 후진 방식 자동 미분을 사용해 테이프에 기록된 연산의 그래디언트를 계산함.
def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        
        print("loss" ,c())
        print(loss)
        grads = tape.gradient(loss, variables)
            # variables = [W, b]
        print("grads")
        return grads
    
print(grad_fn(x_data, y_data))

loss 1
tf.Tensor(0.09595555, shape=(), dtype=float32)
grads
[<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[ 0.00492847,  0.00100384, -0.00593227],
       [ 0.00166355,  0.00089782, -0.00256141],
       [-0.00456257,  0.00075494,  0.00380766],
       [ 0.00132797, -0.00238956,  0.00106162]], dtype=float32)>, None]


In [34]:
def fit(X, Y, epochs = 2000, verbose = 100):  #2000번 반복
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
        # 경사하강법을 사용하세요
    
    for i in range(epochs):
        grads = grad_fn(X, Y)
        if ((i==0) | ((i + 1)%verbose == 0)):
            print(W)
            print("미분값: ")
            print(grads)
        optimizer.apply_gradients(zip(grads, variables))
        if ((i==0) | ((i + 1)%verbose == 0)):
            print('Loss at epoch %d: %f' %(i+1, cost_fn(X, Y).numpy()))
        
fit(x_data, y_data)

loss 2
tf.Tensor(0.09595555, shape=(), dtype=float32)
grads
<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[-6.252283  ,  0.19977517,  7.140748  ],
       [-1.2583793 , -0.40887877,  2.6512926 ],
       [ 6.135336  , -0.8454327 , -3.701212  ],
       [-2.0049138 ,  2.8055422 , -0.724117  ]], dtype=float32)>
미분값: 
[<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[ 0.00492847,  0.00100384, -0.00593227],
       [ 0.00166355,  0.00089782, -0.00256141],
       [-0.00456257,  0.00075494,  0.00380766],
       [ 0.00132797, -0.00238956,  0.00106162]], dtype=float32)>, None]
Loss at epoch 1: 0.095944
loss 3
tf.Tensor(0.09594412, shape=(), dtype=float32)
grads
loss 4
tf.Tensor(0.09593262, shape=(), dtype=float32)
grads
loss 5
tf.Tensor(0.09592111, shape=(), dtype=float32)
grads
loss 6
tf.Tensor(0.095909566, shape=(), dtype=float32)
grads
loss 7
tf.Tensor(0.095897995, shape=(), dtype=float32)
grads
loss 8
tf.Tensor(0.09588661, shape=(), dtype=float32)
grads
loss 9
tf.Ten

loss 52
tf.Tensor(0.0953834, shape=(), dtype=float32)
grads
loss 53
tf.Tensor(0.095371954, shape=(), dtype=float32)
grads
loss 54
tf.Tensor(0.09536064, shape=(), dtype=float32)
grads
loss 55
tf.Tensor(0.09534927, shape=(), dtype=float32)
grads
loss 56
tf.Tensor(0.095337905, shape=(), dtype=float32)
grads
loss 57
tf.Tensor(0.09532654, shape=(), dtype=float32)
grads
loss 58
tf.Tensor(0.09531511, shape=(), dtype=float32)
grads
loss 59
tf.Tensor(0.09530382, shape=(), dtype=float32)
grads
loss 60
tf.Tensor(0.0952924, shape=(), dtype=float32)
grads
loss 61
tf.Tensor(0.095281035, shape=(), dtype=float32)
grads
loss 62
tf.Tensor(0.09526976, shape=(), dtype=float32)
grads
loss 63
tf.Tensor(0.09525836, shape=(), dtype=float32)
grads
loss 64
tf.Tensor(0.095247045, shape=(), dtype=float32)
grads
loss 65
tf.Tensor(0.09523561, shape=(), dtype=float32)
grads
loss 66
tf.Tensor(0.095224306, shape=(), dtype=float32)
grads
loss 67
tf.Tensor(0.09521301, shape=(), dtype=float32)
grads
loss 68
tf.Tensor(0.0

Loss at epoch 100: 0.094818
loss 102
tf.Tensor(0.09481774, shape=(), dtype=float32)
grads
loss 103
tf.Tensor(0.09480646, shape=(), dtype=float32)
grads
loss 104
tf.Tensor(0.094795235, shape=(), dtype=float32)
grads
loss 105
tf.Tensor(0.09478399, shape=(), dtype=float32)
grads
loss 106
tf.Tensor(0.0947728, shape=(), dtype=float32)
grads
loss 107
tf.Tensor(0.09476146, shape=(), dtype=float32)
grads
loss 108
tf.Tensor(0.09475018, shape=(), dtype=float32)
grads
loss 109
tf.Tensor(0.094739065, shape=(), dtype=float32)
grads
loss 110
tf.Tensor(0.09472781, shape=(), dtype=float32)
grads
loss 111
tf.Tensor(0.09471661, shape=(), dtype=float32)
grads
loss 112
tf.Tensor(0.09470538, shape=(), dtype=float32)
grads
loss 113
tf.Tensor(0.09469412, shape=(), dtype=float32)
grads
loss 114
tf.Tensor(0.09468295, shape=(), dtype=float32)
grads
loss 115
tf.Tensor(0.09467177, shape=(), dtype=float32)
grads
loss 116
tf.Tensor(0.09466052, shape=(), dtype=float32)
grads
loss 117
tf.Tensor(0.09464927, shape=(), 

loss 155
tf.Tensor(0.09422515, shape=(), dtype=float32)
grads
loss 156
tf.Tensor(0.094214074, shape=(), dtype=float32)
grads
loss 157
tf.Tensor(0.09420298, shape=(), dtype=float32)
grads
loss 158
tf.Tensor(0.094191864, shape=(), dtype=float32)
grads
loss 159
tf.Tensor(0.09418077, shape=(), dtype=float32)
grads
loss 160
tf.Tensor(0.09416965, shape=(), dtype=float32)
grads
loss 161
tf.Tensor(0.09415853, shape=(), dtype=float32)
grads
loss 162
tf.Tensor(0.094147444, shape=(), dtype=float32)
grads
loss 163
tf.Tensor(0.094136275, shape=(), dtype=float32)
grads
loss 164
tf.Tensor(0.09412533, shape=(), dtype=float32)
grads
loss 165
tf.Tensor(0.09411414, shape=(), dtype=float32)
grads
loss 166
tf.Tensor(0.09410313, shape=(), dtype=float32)
grads
loss 167
tf.Tensor(0.09409201, shape=(), dtype=float32)
grads
loss 168
tf.Tensor(0.094080925, shape=(), dtype=float32)
grads
loss 169
tf.Tensor(0.09406994, shape=(), dtype=float32)
grads
loss 170
tf.Tensor(0.09405881, shape=(), dtype=float32)
grads
los

loss 205
tf.Tensor(0.09367273, shape=(), dtype=float32)
grads
loss 206
tf.Tensor(0.093661815, shape=(), dtype=float32)
grads
loss 207
tf.Tensor(0.093650766, shape=(), dtype=float32)
grads
loss 208
tf.Tensor(0.09363982, shape=(), dtype=float32)
grads
loss 209
tf.Tensor(0.093628824, shape=(), dtype=float32)
grads
loss 210
tf.Tensor(0.09361789, shape=(), dtype=float32)
grads
loss 211
tf.Tensor(0.093606874, shape=(), dtype=float32)
grads
loss 212
tf.Tensor(0.09359591, shape=(), dtype=float32)
grads
loss 213
tf.Tensor(0.09358494, shape=(), dtype=float32)
grads
loss 214
tf.Tensor(0.093574025, shape=(), dtype=float32)
grads
loss 215
tf.Tensor(0.09356308, shape=(), dtype=float32)
grads
loss 216
tf.Tensor(0.0935521, shape=(), dtype=float32)
grads
loss 217
tf.Tensor(0.09354104, shape=(), dtype=float32)
grads
loss 218
tf.Tensor(0.09353011, shape=(), dtype=float32)
grads
loss 219
tf.Tensor(0.093519256, shape=(), dtype=float32)
grads
loss 220
tf.Tensor(0.09350826, shape=(), dtype=float32)
grads
los

...

loss 1795
tf.Tensor(0.07888332, shape=(), dtype=float32)
grads
loss 1796
tf.Tensor(0.07887547, shape=(), dtype=float32)
grads
loss 1797
tf.Tensor(0.07886745, shape=(), dtype=float32)
grads
loss 1798
tf.Tensor(0.0788597, shape=(), dtype=float32)
grads
loss 1799
tf.Tensor(0.07885182, shape=(), dtype=float32)
grads
loss 1800
tf.Tensor(0.07884393, shape=(), dtype=float32)
grads
loss 1801
tf.Tensor(0.07883604, shape=(), dtype=float32)
grads
<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[-7.0536904 ,  0.03372071,  8.108177  ],
       [-1.5306357 , -0.5557517 ,  3.0704167 ],
       [ 6.8827124 , -0.96994627, -4.324079  ],
       [-2.2241018 ,  3.197432  , -0.8968225 ]], dtype=float32)>
미분값: 
[<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[ 0.00404202,  0.00084821, -0.0048902 ],
       [ 0.00137943,  0.00074428, -0.00212367],
       [-0.00379144,  0.00063557,  0.00315596],
       [ 0.00111812, -0.00198947,  0.00087145]], dtype=float32)>, None]
Loss at epoch 1800: 0

loss 1844
tf.Tensor(0.07849923, shape=(), dtype=float32)
grads
loss 1845
tf.Tensor(0.07849148, shape=(), dtype=float32)
grads
loss 1846
tf.Tensor(0.07848367, shape=(), dtype=float32)
grads
loss 1847
tf.Tensor(0.07847589, shape=(), dtype=float32)
grads
loss 1848
tf.Tensor(0.07846803, shape=(), dtype=float32)
grads
loss 1849
tf.Tensor(0.07846029, shape=(), dtype=float32)
grads
loss 1850
tf.Tensor(0.078452446, shape=(), dtype=float32)
grads
loss 1851
tf.Tensor(0.078444704, shape=(), dtype=float32)
grads
loss 1852
tf.Tensor(0.078436896, shape=(), dtype=float32)
grads
loss 1853
tf.Tensor(0.07842916, shape=(), dtype=float32)
grads
loss 1854
tf.Tensor(0.078421295, shape=(), dtype=float32)
grads
loss 1855
tf.Tensor(0.078413606, shape=(), dtype=float32)
grads
loss 1856
tf.Tensor(0.078405805, shape=(), dtype=float32)
grads
loss 1857
tf.Tensor(0.07839794, shape=(), dtype=float32)
grads
loss 1858
tf.Tensor(0.078390196, shape=(), dtype=float32)
grads
loss 1859
tf.Tensor(0.07838248, shape=(), dtype=

loss 1897
tf.Tensor(0.078087956, shape=(), dtype=float32)
grads
loss 1898
tf.Tensor(0.07808031, shape=(), dtype=float32)
grads
loss 1899
tf.Tensor(0.07807253, shape=(), dtype=float32)
grads
loss 1900
tf.Tensor(0.078064844, shape=(), dtype=float32)
grads
loss 1901
tf.Tensor(0.07805711, shape=(), dtype=float32)
grads
<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[-7.0939145 ,  0.0252765 ,  8.156841  ],
       [-1.5443672 , -0.56315696,  3.0915534 ],
       [ 6.920446  , -0.9762717 , -4.3554883 ],
       [-2.2352362 ,  3.2172344 , -0.9054936 ]], dtype=float32)>
미분값: 
[<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[ 0.00400248,  0.00084092, -0.00484337],
       [ 0.00136689,  0.0007369 , -0.00210365],
       [-0.00375516,  0.00062991,  0.00312537],
       [ 0.00110881, -0.00197098,  0.00086229]], dtype=float32)>, None]
Loss at epoch 1900: 0.078049
loss 1902
tf.Tensor(0.078049436, shape=(), dtype=float32)
grads
loss 1903
tf.Tensor(0.07804166, shape=(), dtype=flo

loss 1946
tf.Tensor(0.07771145, shape=(), dtype=float32)
grads
loss 1947
tf.Tensor(0.07770379, shape=(), dtype=float32)
grads
loss 1948
tf.Tensor(0.0776961, shape=(), dtype=float32)
grads
loss 1949
tf.Tensor(0.07768847, shape=(), dtype=float32)
grads
loss 1950
tf.Tensor(0.07768078, shape=(), dtype=float32)
grads
loss 1951
tf.Tensor(0.07767317, shape=(), dtype=float32)
grads
loss 1952
tf.Tensor(0.07766561, shape=(), dtype=float32)
grads
loss 1953
tf.Tensor(0.0776579, shape=(), dtype=float32)
grads
loss 1954
tf.Tensor(0.07765032, shape=(), dtype=float32)
grads
loss 1955
tf.Tensor(0.077642635, shape=(), dtype=float32)
grads
loss 1956
tf.Tensor(0.07763499, shape=(), dtype=float32)
grads
loss 1957
tf.Tensor(0.07762745, shape=(), dtype=float32)
grads
loss 1958
tf.Tensor(0.07761976, shape=(), dtype=float32)
grads
loss 1959
tf.Tensor(0.07761223, shape=(), dtype=float32)
grads
loss 1960
tf.Tensor(0.07760456, shape=(), dtype=float32)
grads
loss 1961
tf.Tensor(0.07759689, shape=(), dtype=float32)

loss 1999
tf.Tensor(0.0773081, shape=(), dtype=float32)
grads
loss 2000
tf.Tensor(0.07730058, shape=(), dtype=float32)
grads
loss 2001
tf.Tensor(0.07729298, shape=(), dtype=float32)
grads
<tf.Variable 'weight:0' shape=(4, 3) dtype=float32, numpy=
array([[-7.133745  ,  0.0169077 ,  8.205035  ],
       [-1.5579683 , -0.57049257,  3.11249   ],
       [ 6.957826  , -0.9825417 , -4.386596  ],
       [-2.2462711 ,  3.2368548 , -0.9140792 ]], dtype=float32)>
미분값: 
[<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[ 0.00396352,  0.00083302, -0.00479656],
       [ 0.00135362,  0.00072993, -0.00208364],
       [-0.00372064,  0.00062408,  0.00309647],
       [ 0.00109848, -0.001953  ,  0.00085444]], dtype=float32)>, None]
Loss at epoch 2000: 0.077285


Prediction Check

In [12]:
sample_data = [[2,1,3,2]]
sample_data = np.asarray(sample_data, dtype = np.float32)

a = hypothesis(sample_data)
# 여기서 가장 높은 예측값을 a로 지정함 
print(a)
print(tf.argmax(a,1))
    # 가장 높은 예측값이 가리키는 값을 뽑는 것 0,1,2 중 2를 뽑음

tf.Tensor([[0.00112886 0.08154673 0.9173244 ]], shape=(1, 3), dtype=float32)
tf.Tensor([2], shape=(1,), dtype=int64)


In [13]:
# 실제 데이터 가지고 해보자
b = hypothesis(x_data)
print(b)
print(tf.argmax(b,1))
    # 결과를 보면 첫번째에서 가장 높은 값 2, 두번째에서 가장 높은 값 2 ... 마지막에서 가장 높은 값 0
    # 이런 식으로 결과가 나오게 됨.
print(tf.argmax(y_data, 1))
    # y랑 비교해보면 같으므로 학습이 정상적으로 되었음을 알 수 있음

tf.Tensor(
[[2.1975952e-06 1.2331170e-03 9.9876475e-01]
 [1.1288594e-03 8.1546687e-02 9.1732442e-01]
 [2.2205539e-07 1.6418624e-01 8.3581358e-01]
 [6.3921816e-06 8.5045439e-01 1.4953922e-01]
 [2.6150808e-01 7.2644734e-01 1.2044534e-02]
 [1.3783246e-01 8.6214006e-01 2.7417480e-05]
 [7.4242145e-01 2.5754160e-01 3.6978410e-05]
 [9.2197549e-01 7.8023903e-02 6.0005692e-07]], shape=(8, 3), dtype=float32)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)
tf.Tensor([2 2 2 1 1 1 0 0], shape=(8,), dtype=int64)


Convert as Class

In [14]:
class softmax_classifer(tf.keras.Model):
    def __init__(self, nb_classes):
        super(softmax_classifer, self).__init__()
        self.W = tf.Variable(tf.random.normal((4, nb_classes)), name = 'weight')
        self.b = tf.Variable(tf.random.normal((nb_classes,)), name = 'bias')
        
    def softmax_regression(self, X):
        return tf.nn.softmax(tf.matmul(X, self.W) + self.b)
    
    def cost_fn(self, X, Y):
        logits = self.softmax_regression(X)
        cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.math.log(logits), axis = 1))
        
    def grad_fn(self, X, Y):
        with tf.GradientTape() as tape:
            cost = self.cost_fn(x_data, y_data)
            grads = tape.gradient(cost, self.variables)
            return grads
        
    def fit(self, X, Y, epochs = 2000, verbose = 500):
        optimizer = tf.keras.optimizers.SGD(learning_rate = 0.1)
        
        for i in range(epochs):
            grads = self.grad_fn(X, Y)
            optimizer.apply_gradients(zip(grads, self.varialbes))
            if((i == 0)|((i + 1) % verbose == 0)):
               print('Loss at epochs %d: %f' %(i+1, self.cost_fn(X, Y).numpy()))
            
model = softmax_classifer(nb_classes)
model.fit(x_data, y_data)

TypeError: Cannot convert value None to a TensorFlow DType.